In [40]:
import pandas as pd
import numpy as np
import string
import spacy
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer

nlp = spacy.load('en_core_web_sm')

In [41]:
starfield_reviews_df = pd.read_csv('starfield_reviews.csv')

In [42]:
# Remove as linhas com valores nulos do DataFrame starfield_reviews_df
starfield_reviews_df.dropna(inplace=True)

# Após a remoção, você pode querer redefinir os índices do DataFrame
starfield_reviews_df.reset_index(drop=True, inplace=True)


In [43]:
starfield_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42524 entries, 0 to 42523
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_id    42524 non-null  int64 
 1   review_text  42524 non-null  object
 2   recommended  42524 non-null  bool  
dtypes: bool(1), int64(1), object(1)
memory usage: 706.1+ KB


In [44]:
starfield_reviews_df['reviews_text'] = starfield_reviews_df['review_text'].apply(lambda x: [token for token in x if token not in string.punctuation and not token.isnumeric()])
starfield_reviews_df['reviews_text'] = starfield_reviews_df['reviews_text'] .apply(lambda x: ''.join(x))

In [45]:
starfield_reviews_df['reviews_text']  = starfield_reviews_df['reviews_text'].apply(lambda x: [token.lemma_.lower() for token in nlp(x) if (token.is_stop == False and len(token.text)>2)])
starfield_reviews_df['reviews_text']  = starfield_reviews_df['reviews_text'].apply(lambda x: ' '.join(x))

In [46]:


X_train, X_test, y_train, y_test = train_test_split(starfield_reviews_df['review_text'], starfield_reviews_df['recommended'], test_size=0.2, random_state=42, stratify=starfield_reviews_df['recommended'])


In [47]:
from gensim.scripts.glove2word2vec import glove2word2vec

# Converta o arquivo GloVe para o formato Word2Vec
glove_input_file = './glove.6B/glove.6B.300d.txt'
word2vec_output_file = 'glove.6B.300d.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)

# Carregue o modelo convertido
from gensim.models import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)


C:\Users\gjque\AppData\Local\Temp\ipykernel_13916\2286485540.py:6: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file, word2vec_output_file)


In [48]:
# Crie uma função para obter o vetor de embedding médio de um texto
def text_to_vector(text):
    words = text.split()
    vectors = []
    for word in words:
        if word in glove_model:
            vectors.append(glove_model[word])
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(glove_model.vector_size)

# Divida os dados em recursos (X) e rótulos (y)
X_train = np.array([text_to_vector(text) for text in X_train])
y_train = y_train
X_test = np.array([text_to_vector(text) for text in X_test])
y_test = y_test


In [49]:
# Crie um pipeline com o SVM como classificador
pipeline = Pipeline([
    ('classifier', SVC(kernel='linear'))  # Classificador SVM
])

# Treine o modelo
pipeline.fit(X_train, y_train)

# Faça previsões no conjunto de teste
y_pred = pipeline.predict(X_test)

In [50]:
# Avalie o desempenho do modelo com lematização e embedding de 300 dimensões
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy:.2f}')

# Exiba um relatório de classificação
report = classification_report(y_test, y_pred)
print('Relatório de Classificação:\n', report)

Acurácia: 0.80
Relatório de Classificação:
               precision    recall  f1-score   support

       False       0.67      0.18      0.28      1918
        True       0.80      0.97      0.88      6587

    accuracy                           0.80      8505
   macro avg       0.74      0.58      0.58      8505
weighted avg       0.77      0.80      0.75      8505



In [39]:
# Avalie o desempenho do modelo sem lematização e embedding de 300 dimensões
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy:.2f}')

# Exiba um relatório de classificação
report = classification_report(y_test, y_pred)
print('Relatório de Classificação:\n', report)

Acurácia: 0.80
Relatório de Classificação:
               precision    recall  f1-score   support

       False       0.67      0.18      0.28      1918
        True       0.80      0.97      0.88      6587

    accuracy                           0.80      8505
   macro avg       0.74      0.58      0.58      8505
weighted avg       0.77      0.80      0.75      8505



In [30]:
# Avalie o desempenho do modelo com lematização e embedding de 100 dimensões
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy:.2f}')

# Exiba um relatório de classificação
report = classification_report(y_test, y_pred)
print('Relatório de Classificação:\n', report)

Acurácia: 0.78
Relatório de Classificação:
               precision    recall  f1-score   support

       False       0.76      0.01      0.02      1918
        True       0.78      1.00      0.87      6587

    accuracy                           0.78      8505
   macro avg       0.77      0.50      0.45      8505
weighted avg       0.77      0.78      0.68      8505



In [14]:
# Avalie o desempenho do modelo sem lematização e embedding de 100 dimensões
accuracy = accuracy_score(y_test, y_pred)
print(f'Acurácia: {accuracy:.2f}')

# Exiba um relatório de classificação
report = classification_report(y_test, y_pred)
print('Relatório de Classificação:\n', report)

Acurácia: 0.78
Relatório de Classificação:
               precision    recall  f1-score   support

       False       0.76      0.01      0.02      1918
        True       0.78      1.00      0.87      6587

    accuracy                           0.78      8505
   macro avg       0.77      0.50      0.45      8505
weighted avg       0.77      0.78      0.68      8505

